In [1]:
import ipywidgets as widgets
from IPython.display import display

# 1) Define the DP solver
def knapsack_dp(weights, values, capacity):
    n = len(weights)
    # dp[i][w] = max value using first i items and capacity w
    dp = [[0]*(capacity+1) for _ in range(n+1)]
    for i in range(1, n+1):
        wt, val = weights[i-1], values[i-1]
        for w in range(capacity+1):
            dp[i][w] = dp[i-1][w]
            if wt <= w:
                dp[i][w] = max(dp[i][w], dp[i-1][w-wt] + val)
    # back-track to find items
    res = dp[n][capacity]
    w = capacity
    items = []
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i-1][w]:
            items.append(i-1)
            w -= weights[i-1]
    return res, list(reversed(items))

# 2) (Optional) Define a simple GA solver
import random
def knapsack_ga(weights, values, capacity, pop_size=50, generations=100, mutation_rate=0.05):
    n = len(weights)
    # initialize population of binary chromosomes
    pop = [[random.randint(0,1) for _ in range(n)] for _ in range(pop_size)]
    def fitness(chrom):
        total_w = sum(w*bit for w,bit in zip(weights, chrom))
        total_v = sum(v*bit for v,bit in zip(values, chrom))
        return total_v if total_w <= capacity else 0

    for _ in range(generations):
        # evaluate and sort
        scored = [(fitness(ch), ch) for ch in pop]
        scored.sort(reverse=True, key=lambda x: x[0])
        pop = [ch for _,ch in scored[:pop_size//2]]  # keep top half
        # reproduce
        while len(pop) < pop_size:
            p1, p2 = random.sample(pop, 2)
            # one-point crossover
            pt = random.randrange(1, n)
            child = p1[:pt] + p2[pt:]
            # mutate
            for i in range(n):
                if random.random() < mutation_rate:
                    child[i] = 1 - child[i]
            pop.append(child)
    best = max(pop, key=fitness)
    return fitness(best), best

# 3) Now set up your widgets and handler
weights_input = widgets.Text(
    value='10, 20, 30', 
    description='Weights:',
    placeholder='e.g. 5,10,15'
)
values_input = widgets.Text(
    value='60, 100, 120', 
    description='Values:',
    placeholder='e.g. 30,50,70'
)
capacity_input = widgets.IntText(
    value=50,
    description='Capacity:'
)
solve_button = widgets.Button(description="Solve Knapsack")
output_area  = widgets.Output()

def on_solve_button_click(b):
    with output_area:
        output_area.clear_output()
        # parse inputs
        try:
            weights = [int(x) for x in weights_input.value.split(',')]
            values  = [int(x) for x in values_input.value.split(',')]
        except ValueError:
            print("❌ Please enter valid comma-separated integers.")
            return
        if len(weights) != len(values):
            print("❌ The number of weights and values must match.")
            return
        cap = capacity_input.value

        # run DP
        dp_val, dp_items = knapsack_dp(weights, values, cap)
        print(f"✅ DP Optimal Value: {dp_val}")
        print(f"   Items: {dp_items}")

        # run GA
        ga_val, ga_chrom = knapsack_ga(weights, values, cap)
        ga_items = [i for i,bit in enumerate(ga_chrom) if bit]
        print(f"🧬 GA Best Value: {ga_val}")
        print(f"   Items: {ga_items} (Chromosome: {ga_chrom})")

solve_button.on_click(on_solve_button_click)

display(weights_input, values_input, capacity_input, solve_button, output_area)


Text(value='10, 20, 30', description='Weights:', placeholder='e.g. 5,10,15')

Text(value='60, 100, 120', description='Values:', placeholder='e.g. 30,50,70')

IntText(value=50, description='Capacity:')

Button(description='Solve Knapsack', style=ButtonStyle())

Output()

In [ ]:
test_cases = [
    # ([], [], 10, 0),
    ([5], [10], 5, 10),
    ([5], [10], 4, 0),
    ([1, 2, 3], [6, 10, 12], 5, 22),  # classic example
    ([2, 3, 4, 5], [3, 4, 5, 6], 5, 7),
]
for w, v, c, exp in test_cases:
    res = solve_knapsack_dp(w, v, c)
    assert res == exp, f"Test failed for weights={w}, values={v}, cap={c}: got {res}, expected {exp}"